In [1]:
# @title
# Import Libraries
import numpy as np
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
import plotly.figure_factory as FF
from plotly import tools
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime

from pathlib import Path
import os

warnings.filterwarnings("ignore")

In [2]:
os.chdir(str(Path("../")))

In [3]:
df = pd.read_parquet(Path("data/01_raw/MELBOURNE_HOUSE_PRICES_LESS.pq"))

In [4]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Postcode,Regionname,Propertycount,Distance,CouncilArea
0,Abbotsford,49 Lithgow St,3,h,1490000.0,S,Jellis,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
1,Abbotsford,59A Turner St,3,h,1220000.0,S,Marshall,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
2,Abbotsford,119B Yarra St,3,h,1420000.0,S,Nelson,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
3,Aberfeldie,68 Vida St,3,h,1515000.0,S,Barry,1/04/2017,3040,Western Metropolitan,1543,7.5,Moonee Valley City Council
4,Airport West,92 Clydesdale Rd,2,h,670000.0,S,Nelson,1/04/2017,3042,Western Metropolitan,3464,10.4,Moonee Valley City Council


In [5]:
(df.isnull()
   .mean()
   .sort_values(ascending=False)
   .reset_index()
   .rename(columns = {'index' : "Feature", 0 : 'Missing (%)'})
   .style
   .format(subset = 'Missing (%)', formatter = "{:.2%}" )
)

,Feature,Missing (%)
0,Price,23.15%
1,Suburb,0.00%
2,Address,0.00%
3,Rooms,0.00%
4,Type,0.00%
5,Method,0.00%
6,SellerG,0.00%
7,Date,0.00%
8,Postcode,0.00%
9,Regionname,0.00%


In [6]:
df = df.dropna()


In [7]:
df['Date'] = pd.to_datetime(df['Date'], format = "%d/%m/%Y")

In [8]:
REGIONS = list(df['Regionname'].unique())
REGION_COLOR = dict(zip(REGIONS, ["#2E9AFE", "#FA5858", "#81F781", "#BE81F7", "#FE9A2E", "#04B4AE", "#088A08", "#8A0886"]))

def complementaryColor(my_hex):
    """Returns complementary RGB color

    Example:
    >>>complementaryColor('FFFFFF')
    '000000'
    """
    if my_hex[0] == '#':
        my_hex = my_hex[1:]
    rgb = (my_hex[0:2], my_hex[2:4], my_hex[4:6])
    comp = ['%02X' % (255 - int(a, 16)) for a in rgb]
    return ''.join(comp)

def create_histogram_with_mean(data, name, marker_color):
  mean_price = np.mean(data)
  v, b = np.histogram(data, bins = 'auto')
  ylim = (np.max(v) / np.sum(v)) * 102
  xbins = go.XBins(start=b[0], end=b[-1], size=b[1] - b[0])  # Adjust as needed

  histogram_plot = go.Histogram(
      x=data,
      histnorm='percent',
      xbins = xbins,
      name=name,
      marker=dict(
          color=marker_color
      )
  )

  mean_line = go.Scatter(
      x=[mean_price, mean_price],
      y=[0, ylim],  # Set y-coordinates to span the entire histogram height
      mode='lines',
      name='Mean Price',
      line=dict(
          color= "#" + complementaryColor(marker_color),
          width=2,
          dash='dash'
      )
  )

  return [histogram_plot, mean_line]

overall_plots = create_histogram_with_mean(df['Price'].values, 'All Regions', '#6E6E6E')

region_plot = []
for region in REGIONS:
  region_plot.append(create_histogram_with_mean(df.query("Regionname == @region")['Price'].values, region, REGION_COLOR[region]))

gaussian_distribution_plot = create_histogram_with_mean(np.log(df['Price'].values), 'Log Price (All Regions)', '#800000')

suptitles = ['Overall Price Distribution'] + REGIONS + ['Distribution of Log Price']

fig = tools.make_subplots(rows=6, cols=2, print_grid=False, specs=[[{'colspan': 2}, None], [{}, {}], [{}, {}], [{}, {}], [{}, {}], [{'colspan': 2}, None]],
                         subplot_titles=suptitles)

fig.add_traces(overall_plots, 1, 1)

row_index = 2
col_index = 1
for p in region_plot:
  fig.add_traces(p, row_index, col_index)
  if col_index == 2:
    row_index += 1
    col_index = 0
  col_index += 1


fig.add_traces(gaussian_distribution_plot, 6, 1)

fig['layout'].update(showlegend=False, title="Price Distributions by Region",
                    height=1340, width=960)
iplot(fig, filename='custom-sized-subplot-with-subplot-titles')

In [9]:
weekly_averages = df.groupby(['Regionname', pd.Grouper(key = 'Date', freq='m')])['Price'].mean().reset_index().query("Date < '2018/06/01'")
overall_averages = df.groupby(pd.Grouper(key = 'Date', freq='m'))['Price'].mean().reset_index().query("Date < '2018/06/01'")

fig = go.Figure()

for region in weekly_averages['Regionname'].unique():
    region_data = weekly_averages[weekly_averages['Regionname'] == region]
    fig.add_trace(go.Scatter(
        x=region_data['Date'],
        y=region_data['Price'],
        name=region,
        mode='lines+markers',
        line=dict(color=REGION_COLOR[region], width = 2)  # Differentiate colors
    ))

fig.add_trace(go.Scatter(
    x=overall_averages['Date'],
    y=overall_averages['Price'],
    name='Overall Average Price',
    mode='lines+markers',
    line=dict(color='royalblue', width=2)  # Highlight overall line
))

fig.update_layout(
    title='Average Price per Week by Region',
    xaxis_title='Week Number',
    yaxis_title='Average Price',
    legend_title_text='Region',
    legend_title_font_size=16,
    paper_bgcolor='rgb(255, 255, 224)',
    plot_bgcolor='rgb(255, 255, 246)',
)

fig.show()


In [10]:
df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y")
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

In [11]:
MONTH_LABELS  = ['January', 'February', 'March', 'April',
          'May', 'June', 'July', 'August', 'September',
          'October', 'November', 'December']


colors = ['#ffb4da', '#b4b4ff', '#daffb4', '#fbab60', '#fa8072', '#FA6006',
          '#FDB603', '#639702', '#dacde6', '#faec72', '#9ab973', '#87cefa']


def month_year_sales(df, month, year):
    double_conditional = df['Price'].loc[(df['Month'] == month) & (df['Year'] == year)].sum()
    return double_conditional

df_filtered = df[(df['Year'] <= 2017) | ((df['Year'] == 2018) & (df['Month'] <= 5))]

YEAR_COLOR = dict(zip(sorted(df_filtered['Year'].unique()), ['rgba(0, 128, 128, 0.95)', 'rgba(255, 72, 72, 0.95)','rgba(72, 255, 72, 0.95)']))
YEAR_COLOR_MARKER = dict(zip(sorted(df_filtered['Year'].unique()), ['rgba(0, 74, 147, 1)', 'rgba(219, 0, 0, 1)','rgba(0, 147, 74, 1)']))

sales_data_per_month = {}
plots = []

for year in sorted(df['Year'].unique()):
  print()
  sales_data_per_month[year] = [month_year_sales(df_filtered, int(m), int(year)) if m in df_filtered.query("Year == @year")['Month'].unique() else 0 for m in range(1,13)]
  plots.append(go.Scatter(
      x=sales_data_per_month[year],
      y=MONTH_LABELS,
      xaxis='x2',
      yaxis='y2',
      mode='markers',
      name=str(year),
      marker=dict(
          color= YEAR_COLOR[year],
          line=dict(
              color='rgba(56, 56, 56, 1)',
              width=1.5,
          ),
          symbol='circle',
          size=16,
        )
    )
  )

layout = go.Layout(
    title="Sales by Month for the Years <br>(2016, 2017, 2018)",
    xaxis=dict(
        showgrid=True,
        showline=True,
        linecolor='rgb(102, 102, 102)',
        titlefont=dict(
            color='rgb(204, 204, 204)'
        ),
        tickfont=dict(
            color='rgb(102, 102, 102)',
        ),
        # autotick=False,
        dtick=10,
        ticks='outside',
        tickcolor='rgb(102, 102, 102)',
    ),
    margin=dict(
        l=140,
        r=40,
        b=50,
        t=80
    ),
    legend=dict(
        font=dict(
            size=10,
        ),
        yanchor='top',
        xanchor='left',
    ),
    width=800,
    height=600,
    paper_bgcolor='rgb(255, 255, 224)',
    plot_bgcolor='rgb(255, 255, 246)',
    hovermode='closest',
    xaxis2=dict(
        showgrid=True,
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='rgb(102, 102, 102)'
    )
)

fig = go.Figure(data=plots, layout=layout)
iplot(fig, filename='multiple-subplots')

In [12]:

import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [13]:
df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Postcode', 'Regionname', 'Propertycount', 'Distance',
       'CouncilArea', 'Month', 'Year'],
      dtype='object')

In [17]:
features = ['Distance', 'Year', 'Month', 'Rooms', 'Regionname']
categorical_features = ['Regionname']
target = 'Price'
test_size = 0.3

In [16]:
df = df[features + [target]]
df_train, df_test = train_test_split(df, test_size = test_size)

In [22]:
model_query = f"{target} ~ " + " + ".join([f"C({c})" for c in categorical_features]) + " + " + " + ".join([f for f in features if f not in categorical_features])
model = smf.ols(model_query, df_train).fit()

results = model.predict(df_test)

metrics = dict(model.params, **{'Erro' : results})

In [24]:
dict(model.params)

{'Intercept': -91202025.96883327,
 'C(Regionname)[T.Eastern Victoria]': 225187.77707391555,
 'C(Regionname)[T.Northern Metropolitan]': -246869.6693387686,
 'C(Regionname)[T.Northern Victoria]': 139670.99726600258,
 'C(Regionname)[T.South-Eastern Metropolitan]': 155031.92216283252,
 'C(Regionname)[T.Southern Metropolitan]': 237781.47155300682,
 'C(Regionname)[T.Western Metropolitan]': -329325.9212947028,
 'C(Regionname)[T.Western Victoria]': -77877.91661245824,
 'Distance': -36182.31131063517,
 'Year': 45430.24668575359,
 'Month': 4304.547407791205,
 'Rooms': 342749.1360965707}

In [26]:
results = model.predict(df_test)


In [28]:
y_hat = model.predict(df_test)
mean_absolute_error(y_hat, df_test[target])

285489.937039693

In [ ]:
from src.simple_pipeline.shared_code.preprocessing import * 
from src.simple_pipeline.pipelines.modeling.nodes import *